# Data shaping

## Setup

In [ ]:
import utils
import polars as pl
from pathlib import Path

In [ ]:
for path in Path("input").glob("*.accdb"):
    df = utils.get_df(path)

In [ ]:
# define identifier columns
expr_identifier = pl.col("DISTRICT", "LOCATION OF SCREENING", "DATESCREEN", "ICNUMBER")

## 1 - `ICNUMBER` vs. `GENDER` for full I/C number patients

In [ ]:
df.select(expr_identifier, "GENDER CODE").filter(
  (pl.col("ICNUMBER").str.len_chars() == 12) & (pl.col("ICNUMBER").str.contains(r"\d$"))
).with_columns(
  pl.col("GENDER CODE").cast(pl.Int16) % 2,
  pl.col("ICNUMBER").str.slice(-1).cast(pl.Int16).alias("R1_IC_tail") % 2,
).filter(pl.col("GENDER CODE") != pl.col("R1_IC_tail"))


## 2 - `LESION` vs. `REFERAL TO SPECIALIST`

In [ ]:
df.select(
  expr_identifier, "LESION", "REFERAL TO SPECIALIST",
  pl.when(pl.col("LESION") != pl.col("REFERAL TO SPECIALIST"))
  .then(1)
  .otherwise(0)
  .alias("R2_violation"),
).filter(pl.col("R2_violation") == 1)
